# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900299  89.199585  130.451974  89.487615  127.713291   
2022-12-27  128.097504  87.332367  129.456999  88.891696  126.806975   
2022-12-28  124.166786  85.872353  129.082622  87.918349  123.999315   
2022-12-29  127.683723  88.345436  128.540789  88.757616  125.831667   
2022-12-30  127.998978  88.126923  128.018685  88.226241  125.536140   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  87.024475  128.974267  87.024475  63814900  17815000  
2022-12-27  86.940055  129.427446  88.702985  69007800  15470900  
2022-12-28  85.782968  127.742834  86.905285  85438400  17879600  
2022-12-29  86.398758  126.087797  86.438487  75703700  18280700  
2022-12-30  86.438473  126.501579  86.771195  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

Strategy started!
Bought 1239 AAPL and 945 GOOG
Strategy finished!


In [5]:
result.head()

,ITERATION,BALANCE,POSITIONS_TOTAL,EQUITY
0,2018-01-02,37.190189,99962.809811,100000.000000
1,2018-01-03,37.190189,100774.440376,100811.630566
2,2018-01-04,37.190189,101190.539085,101227.729275
3,2018-01-05,37.190189,102505.058304,102542.248493
4,2018-01-08,37.190189,102537.476349,102574.666538


## <b> Backtest Statistics </b>

In [6]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 37.19
Final equity: 241907.87
---------------------------


In [7]:
bt.statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'avg_abs_return': np.float64(141907.8653755188),
 'avg_abs_return_per_win': np.float64(141907.8653755188),
 'avg_abs_return_per_lost': nan,
 'median_abs_return': np.float64(141907.8653755188),
 'median_abs_return_per_win': np.float64(141907.8653755188),
 'median_abs_return_per_lost': nan,
 'std_return': np.float64(nan),
 'mad_return': np.float64(0.0),
 'downside_std': np.float64(0.0),
 'upside_std': np.float64(141907.8653755188),
 'skewness': np.float64(nan),
 'kurtosis': np.float64(nan),
 'mean_median_gap': np.float64(0.0),
 'avg_return_over_vol': np.float64(nan),
 'median_return_over_vol': np.float64(nan)}

In [8]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,REASON_CLOSED,POSITIONS_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,32b33f71-07cf-48e4-b1e8-d26c40deb24a,OPEN,Buy and Hold Trade,141907.865376,2018-01-02,None,None,241870.675186,None,None,None


In [9]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,TRADE_ID,REQUESTED_ITERATION,REQUESTED_QUANTITY,REQUESTED_PRICE,REQUESTED_FEE,REQUESTED_GROSS_TOTAL,REQUESTED_TOTAL,FILLED_ITERATION,FILLED_QUANTITY,FILLED_PRICE,FILLED_FEE,FILLED_GROSS_TOTAL,FILLED_TOTAL
0,7ff2bbcc-1333-4898-9c05-206e7978ea0a,AAPL,BUY,FILLED,304483bd-3f68-4bc5-84cf-82c4c4f34c30,2018-01-02,1239,40.341881,0.0,49983.590309,49983.590309,2018-01-02,1239,40.341881,0.0,-49983.590309,-49983.590309
1,d8a83f32-cb87-47df-abdd-72aac616f663,GOOG,BUY,FILLED,304483bd-3f68-4bc5-84cf-82c4c4f34c30,2018-01-02,945,52.888069,0.0,49979.225349,49979.225349,2018-01-02,945,52.888069,0.0,-49979.225349,-49979.225349


In [10]:
bt.strategy.positions

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.70496368408)},
 'GOOG': {'quantity': 945, 'value': np.float64(83279.9490737915)}}